In [ ]:
import numpy.random as nr
from random import shuffle
import math
import shutil
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from object_detection.utils import label_map_util
from matplotlib import pyplot as plt
from CNNRobotLocalisation.Utils.file_utils import *

In [ ]:
PATH = '/media/data/LocalizationDataNew/Output/ValidationTFRecord'
DISTR_PATH = '/media/data/LocalizationDataNew/Remote/data/tmp/ValidationDistraction'
OUT_PATH = '/media/data/LocalizationDataNew/Output/ValidationTFRecordDistr'

In [ ]:
def getBBs(f):
    data = parseXML(f)['annotation']
    xmins, ymins, xmaxs, ymaxs = [],[],[],[]
    for obj in data['object']:
        xmins.append(float(obj['bndbox']['xmin']))
        ymins.append(float(obj['bndbox']['ymin']))
        xmaxs.append(float(obj['bndbox']['xmax']))
        ymaxs.append(float(obj['bndbox']['ymax']))
    return xmins, ymins, xmaxs, ymaxs


def overlap(a,b):
    x = max(a[0], b[0])
    y = max(a[1], b[1])
    w = min(a[0]+a[2], b[0]+b[2]) - x
    h = min(a[1]+a[3], b[1]+b[3]) - y
    if w<0 or h<0: 
        return False
    else:
        return True

def isColliding(x,y,w,h, bbs):
    for i in range(len(bbs[0])):
        xmin = bbs[0][i]-10
        ymin = bbs[1][i]-10
        xmax = bbs[2][i]+10
        ymax = bbs[3][i]+10
        if overlap((xmin,ymin,xmax-xmin,ymax-ymin),(x,y,w,h)):
            return True
    return False

In [ ]:
fs = get_recursive_file_list(PATH, file_extensions=['.xml'])
dfs = get_recursive_file_list(DISTR_PATH, file_extensions=['.png'])
for f in tqdm(fs):
    bbs = getBBs(f) 
    bg = Image.open(f.replace('.xml','.jpg')).convert('RGB')
    for i in range(3):
        df = nr.choice(dfs)
        try:
            obj = Image.open(df)
        except OSError as e:
            print(e)
        #scl = int(nr.uniform(10, 200))
        #obj = obj.resize((scl,scl), resample=Image.LANCZOS)
        rot = nr.randint(360)
        obj = obj.rotate(rot, resample=Image.BICUBIC, expand=True)
        obj=obj.crop(obj.getbbox())
        pos = (nr.randint(0, bg.width-obj.width), nr.randint(0, bg.height-obj.height))
        if isColliding(pos[0],pos[1],obj.width,obj.height,bbs):
            continue
        bg.paste(obj, pos, obj.split()[-1])
    out_f = f.replace(PATH,OUT_PATH).replace('.xml','.jpg')
    os.makedirs(out_f.rsplit('/',1)[0], exist_ok=True)
    bg.save(out_f)    
    shutil.copy(f, f.replace(PATH,OUT_PATH))